# Singleton Networks

In [188]:
import qualreas as qr
import os
import copy

In [189]:
qr_path = os.path.join(os.getenv('PYPROJ'), 'qualreas')
alg_dir = os.path.join(qr_path, "Algebras")

## Make a Test Network

In [190]:
test1_net_dict = {
    'name': 'Network Copy Test #1',
    'algebra': 'Extended_Linear_Interval_Algebra',
    'description': 'Testing/Developing network copy functionality',
    'nodes': [
        ['U', ['ProperInterval', 'Point']],
        ['V', ['ProperInterval', 'Point']],
        ['W', ['ProperInterval']],
        ['X', ['Point']]
    ],
    'edges': [
        ['U', 'V', 'B'],
        ['U', 'W', 'M'],
        ['W', 'V', 'O'],
        ['X', 'W', 'D']
    ]
}

In [191]:
test2_net_dict = {
    'name': 'Network Copy Test #2',
    'algebra': 'Extended_Linear_Interval_Algebra',
    'description': 'Testing/Developing network copy functionality',
    'nodes': [
        ['X', ['ProperInterval']],
        ['Y', ['ProperInterval']],
        ['Z', ['ProperInterval']]
    ],
    'edges': [
        ['X', 'Y', 'B'],
        ['Y', 'Z', 'B']
    ]
}

In [192]:
test1_net = qr.Network(algebra_path=alg_dir, network_dict=test1_net_dict)

In [193]:
test2_net = qr.Network(algebra_path=alg_dir, network_dict=test2_net_dict)

In [194]:
test1_net.propagate()
test1_net.summary()


Network Copy Test #1: 4 nodes, 16 edges
  Algebra: Extended_Linear_Interval_Algebra
  U:['ProperInterval']
    => U: E
    => V: B
    => W: M
    => X: B
  V:['ProperInterval']
    => V: E
    => U: BI
    => W: OI
    => X: BI|DI|PSI
  W:['ProperInterval']
    => W: E
    => U: MI
    => V: O
    => X: DI
  X:['Point']
    => X: PE
    => W: D
    => U: BI
    => V: B|D|PS


In [195]:
test2_net.propagate()
test2_net.summary()


Network Copy Test #2: 3 nodes, 9 edges
  Algebra: Extended_Linear_Interval_Algebra
  X:['ProperInterval']
    => X: E
    => Y: B
    => Z: B
  Y:['ProperInterval']
    => Y: E
    => X: BI
    => Z: B
  Z:['ProperInterval']
    => Z: E
    => Y: BI
    => X: BI


### Test Short Version of Network Summary

In [196]:
test1_net.summary(show_all=False)


Network Copy Test #1: 4 nodes, 16 edges
  Algebra: Extended_Linear_Interval_Algebra
  U:['ProperInterval']
    => U: E
    => V: B
    => W: M
    => X: B
  V:['ProperInterval']
    => V: E
    => W: OI
    => X: BI|DI|PSI
  W:['ProperInterval']
    => W: E
    => X: DI
  X:['Point']
    => X: PE


## Test Changing Constraint on an Edge

Look at all the edge contraints

In [197]:
for eg in test1_net.edges:
    print(test1_net.edges[eg[0], eg[1]]['constraint'])

E
B
M
B
E
BI
OI
BI|DI|PSI
E
MI
O
DI
PE
D
BI
B|D|PS


Grab the Head (src) and Tail (tgt) of the 3rd edge, above.

In [198]:
src, tgt = list(test1_net.edges)[2]
test1_net.edges[src,tgt]['constraint']

relset(['M'])

Change the constraint and look at the result on the edge & its converse.

In [199]:
test1_net.set_constraint(src, tgt, test1_net.algebra.relset('D|M|FI'))

In [200]:
test1_net.edges[src,tgt]['constraint']

relset(['D', 'FI', 'M'])

In [201]:
test1_net.edges[tgt,src]['constraint']

relset(['DI', 'F', 'MI'])

## Test Copy Network

In [202]:
test1_net_copy = test1_net.copy()
#test1_net_copy = qr.copy(test1_net)

In [203]:
test1_net_copy.summary()


Network Copy Test #1: 4 nodes, 16 edges
  Algebra: Extended_Linear_Interval_Algebra
  U:['ProperInterval']
    => U: E
    => V: B
    => W: D|FI|M
    => X: B
  V:['ProperInterval']
    => V: E
    => U: BI
    => W: OI
    => X: BI|DI|PSI
  W:['ProperInterval']
    => W: E
    => U: DI|F|MI
    => V: O
    => X: DI
  X:['Point']
    => X: PE
    => U: BI
    => V: B|D|PS
    => W: D


In [204]:
done = []
result = []
for eg in test1_net.edges:
    src = eg[0]; tgt = eg[1]
    srcID = src.name; tgtID = tgt.name
    if not (src, tgt) in done:
        cons = test1_net.edges[src, tgt]['constraint']
        print(srcID, tgtID, cons)
        if len(cons) > 1:
            result.append((srcID, tgtID, cons))
    done.append((tgt, src))

U U E
U V B
U W D|FI|M
U X B
V V E
V W OI
V X BI|DI|PSI
W W E
W X DI
X X PE


In [205]:
rels = []
for rel in result[0][2]:
    rels.append(rel)

In [206]:
rels

['D', 'FI', 'M']

In [211]:
def generate_singletons(in_work, result):
    if len(in_work) == 0:
        print("DONE")
        return result
    else:
        print("Get next net in work")
        next_net = in_work.pop()
        if finished(next_net):
            print("   This one's finished")
            result.append(next_net)
            generate_singletons(in_work, result)
        else:
            print("   Expanding net")
            generate_singletons(in_work + expand(next_net), result)

def expand(net):
    expansion = []
    for src, tgt in net.edges:
        edge_constraint = net.edges[src, tgt]['constraint']
        if len(edge_constraint) > 1:
            print("--------")
            print(edge_constraint)
            for rel in edge_constraint:
                print(rel)
                net_copy = net.copy()
                src_node, tgt_node, _ = net_copy.get_edge_by_names(src.name, tgt.name, return_names=False)
                net_copy.set_constraint(src_node, tgt_node, net_copy.algebra.relset(rel))
                expansion.append(net_copy)
                print(expansion)
            break
    return expansion

def finished(net):
    """Returns True if all constraints are singletons."""
    answer = True
    for src, tgt in net.edges:
        edge_constraint = net.edges[src, tgt]['constraint']
        if len(edge_constraint) > 1:
            answer = False
            break
    return answer

In [220]:
x = generate_singletons([test1_net], list())

Get next net in work
   Expanding net
--------
D|FI|M
D
FI
[<qualreas.Network object at 0x7fc2e9ae7510>, <qualreas.Network object at 0x7fc2e9ae7c10>]
M
[<qualreas.Network object at 0x7fc2e9ae7510>, <qualreas.Network object at 0x7fc2e9ae7c10>, <qualreas.Network object at 0x7fc2e93052d0>]
Get next net in work
   Expanding net
--------
BI|DI|PSI
BI
DI
[<qualreas.Network object at 0x7fc2e94487d0>, <qualreas.Network object at 0x7fc2e8fe7e50>]
PSI
[<qualreas.Network object at 0x7fc2e94487d0>, <qualreas.Network object at 0x7fc2e8fe7e50>, <qualreas.Network object at 0x7fc3090e2b50>]
Get next net in work
   This one's finished
Get next net in work
   This one's finished
Get next net in work
   This one's finished
Get next net in work
   Expanding net
--------
BI|DI|PSI
BI
DI
[<qualreas.Network object at 0x7fc3092e9950>, <qualreas.Network object at 0x7fc2e92b7290>]
PSI
[<qualreas.Network object at 0x7fc3092e9950>, <qualreas.Network object at 0x7fc2e92b7290>, <qualreas.Network object at 0x7fc3092

In [221]:
len(x)

TypeError: object of type 'NoneType' has no len()

In [214]:
foo = expand(test1_net)
foo

--------
D|FI|M
D
FI
[<qualreas.Network object at 0x7fc318a5fed0>, <qualreas.Network object at 0x7fc2e93ccd50>]
M
[<qualreas.Network object at 0x7fc318a5fed0>, <qualreas.Network object at 0x7fc2e93ccd50>, <qualreas.Network object at 0x7fc2e8fe74d0>]


In [215]:
foo[0].summary(show_all=False)


Network Copy Test #1: 4 nodes, 16 edges
  Algebra: Extended_Linear_Interval_Algebra
  U:['ProperInterval']
    => U: E
    => V: B
    => W: D
    => X: B
  V:['ProperInterval']
    => V: E
    => W: OI
    => X: BI|DI|PSI
  W:['ProperInterval']
    => W: E
    => X: DI
  X:['Point']
    => X: PE


In [216]:
foo[1].summary(show_all=False)


Network Copy Test #1: 4 nodes, 16 edges
  Algebra: Extended_Linear_Interval_Algebra
  U:['ProperInterval']
    => U: E
    => V: B
    => W: FI
    => X: B
  V:['ProperInterval']
    => V: E
    => W: OI
    => X: BI|DI|PSI
  W:['ProperInterval']
    => W: E
    => X: DI
  X:['Point']
    => X: PE


In [217]:
foo[2].summary(show_all=False)


Network Copy Test #1: 4 nodes, 16 edges
  Algebra: Extended_Linear_Interval_Algebra
  U:['ProperInterval']
    => U: E
    => V: B
    => W: M
    => X: B
  V:['ProperInterval']
    => V: E
    => W: OI
    => X: BI|DI|PSI
  W:['ProperInterval']
    => W: E
    => X: DI
  X:['Point']
    => X: PE


In [218]:
finished(test1_net)

False

In [219]:
finished(test2_net)

True